<a href="https://colab.research.google.com/github/rauledop1/TEWS/blob/main/Catastro_BN_chileipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
%pip install leafmap rarfile lonboard
#%pip install lonboard

In [42]:
import duckdb
import leafmap
import rarfile
import requests
import os
import pandas as pd
import shutil

In [43]:
#download file function
def download_file(url, output_path):
    # Send a HTTP request to the specified URL
    response = requests.get(url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Open the output file in binary mode and write the content
        with open(output_path, 'wb') as file:
            file.write(response.content)
        print(f"Downloaded file to {output_path}")
    else:
        print(f"Failed to download file. Status code: {response.status_code}")

#uncompress rar file
def extract_rar(rar_file, extract_dir):
    # Open the rar file
    with rarfile.RarFile(rar_file) as rf:
        # Extract all contents to the specified directory
        rf.extractall(extract_dir)
        print(f"Extracted {rar_file} to {extract_dir}")

def extract_table_to_dataframe(url):
    # Send an HTTP request to the URL
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Use pandas to read the HTML and extract tables
        tables = pd.read_html(response.text)

        # If tables were found, return the first table as a DataFrame
        if tables:
            df = tables[0]
            print(f"Extracted table with {df.shape[0]} rows and {df.shape[1]} columns")
            return df
        else:
            print("No tables found on the webpage.")
            return None
    else:
        print(f"Failed to fetch the webpage. Status code: {response.status_code}")
        return None

def find_shp_files(base_dir):
    shp_files = []

    # Walk through the directory and subdirectories
    for root, dirs, files in os.walk(base_dir):
        # Search for .shp files
        for file in files:
            if file.endswith('.shp'):
                shp_path = os.path.join(root, file)
                shp_files.append(shp_path)
                print(f"Found shapefile: {shp_path}")

    if shp_files:
        print(f"Found {len(shp_files)} shapefile(s).")
        return shp_files
    else:
        print("No shapefiles found.")
        return None


In [44]:
admin_url='https://raw.githubusercontent.com/rauledop1/TEWS/refs/heads/main/Catastro_BN.csv'

In [45]:
df=pd.read_csv(admin_url)

In [46]:
#df=df.query("cod_region == 'CL-MA'")
url_catastro=df.file_url.to_list()
year_update=df.year_update.to_list()
cod_region=df.cod_region.to_list()
crs_file=df.crs.to_list()

In [47]:
#Create a temporal DB
con = duckdb.connect()

#Load spatial and http extensions
con.sql('''
install httpfs;
load httpfs;
INSTALL spatial;
LOAD spatial;
''')

In [ ]:
for i,j,k,c in zip(url_catastro, year_update,cod_region, crs_file):

  compress_file=f'{k}_{j}.rar'
  folder_p=f'{k}_{j}_BN'
  table_name=k.replace('-','_')+f'_{j}'

  download_file(i,compress_file)

  if not os.path.exists(folder_p):
    os.makedirs(folder_p)

  extract_rar(compress_file, folder_p)

  shp=find_shp_files(folder_p)

  shp=shp[0]

  con.sql(f'''
    create table '{table_name}' as
    SELECT *, '{table_name}' as cod_catastro, st_transform(geom, '{c}','EPSG:4326') as geo_geom
      from st_read('{shp}')
    ''')
  os.remove(compress_file)
  shutil.rmtree(folder_p)

Downloaded file to CL-AI_2011.rar
Extracted CL-AI_2011.rar to CL-AI_2011_BN
Found shapefile: CL-AI_2011_BN/11. Región de Aysén/Catastro_RV_R11_2011_v2.shp
Found 1 shapefile(s).


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Downloaded file to CL-AN_2019.rar
Extracted CL-AN_2019.rar to CL-AN_2019_BN
Found shapefile: CL-AN_2019_BN/CBN_REG02_2019/CBN_REG02_2019.shp
Found 1 shapefile(s).


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Downloaded file to CL-AP_2015.rar
Extracted CL-AP_2015.rar to CL-AP_2015_BN
Found shapefile: CL-AP_2015_BN/catastro_rv_r15_2015_v2/catastro_rv_r15_2015_v2.shp
Found 1 shapefile(s).
Downloaded file to CL-AT_2018.rar
Extracted CL-AT_2018.rar to CL-AT_2018_BN
Found shapefile: CL-AT_2018_BN/catastro_rv_r03_2018/catastro_rv_r03_2018.shp
Found 1 shapefile(s).


...

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Downloaded file to CL-LI_2013.rar
Extracted CL-LI_2013.rar to CL-LI_2013_BN
Found shapefile: CL-LI_2013_BN/06. Región de OHiggins/Catastro_RV_R06_2013.shp
Found 1 shapefile(s).


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Downloaded file to CL-LL_2018.rar
Extracted CL-LL_2018.rar to CL-LL_2018_BN
Found shapefile: CL-LL_2018_BN/cbn_10reg_2018/cbn_10reg_2018.shp
Found 1 shapefile(s).


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
con.sql('''
SHOW ALL TABLES
''')

In [ ]:
con.sql('''
create table catastro_bn as
with aa as (
SELECT cod_catastro, id_uso, id_subuso, id_estruc,id_esp1, id_esp2, id_esp3, id_esp4, id_esp5, id_esp6, uso_tierra, uso, subuso, estructura, cobertura, altura, tipo_fores, subtipofor, especi1_ci, especi2_ci, especi3_ci, especi4_ci, especi5_ci, especi6_ci, codreg, codprov, codcom, superf_ha,  geo_geom as geom FROM CL_NB_2015
union
SELECT cod_catastro, id_uso, id_subuso, id_estruc,id_esp1, id_esp2, id_esp3, id_esp4, id_esp5, id_esp6, uso_tierra, uso, subuso, estructura, cobertura, altura, tipo_fores, subtipofor, especi1_ci, especi2_ci, especi3_ci, especi4_ci, especi5_ci, especi6_ci, codreg, codprov, codcom, superf_ha,  geo_geom as geom FROM CL_AI_2011
union
SELECT cod_catastro, id_uso, id_subuso, id_estruc,id_esp1, id_esp2, id_esp3, id_esp4, id_esp5, id_esp6, uso_tierra, uso, subuso, estructura, cobertura, altura, tipo_fores, subtipofor, especi1_ci, especi2_ci, especi3_ci, especi4_ci, especi5_ci, especi6_ci, codreg, codprov, codcom, superf_ha,  geo_geom as geom FROM CL_AN_2019
union
SELECT cod_catastro, id_uso, id_subuso, id_estruc,id_esp1, id_esp2, id_esp3, id_esp4, id_esp5, id_esp6, uso_tierra, uso, subuso, estructura, cobertura, altura, tipo_fores, subtipofor, especi1_ci, especi2_ci, especi3_ci, especi4_ci, especi5_ci, especi6_ci, codreg, codprov, codcom, superf_ha,  geo_geom as geom FROM CL_AP_2015
union
SELECT cod_catastro, id_uso, id_subuso, id_estruc,id_esp1, id_esp2, id_esp3, id_esp4, id_esp5, id_esp6, uso_tierra, uso, subuso, estructura, cobertura, altura, tipo_fores, subtipofor, especi1_ci, especi2_ci, especi3_ci, especi4_ci, especi5_ci, especi6_ci, codreg, codprov, codcom, superf_ha,  geo_geom as geom FROM CL_AT_2018
union
SELECT cod_catastro, id_uso, id_subuso, id_estruc,id_esp1, id_esp2, id_esp3, id_esp4, id_esp5, id_esp6, uso_tierra, uso, subuso, estructura, cobertura, altura, tipo_fores, subtipofor, especi1_ci, especi2_ci, especi3_ci, especi4_ci, especi5_ci, especi6_ci, codreg, codprov, codcomun as codcom, superf_ha,  geo_geom as geom FROM CL_BI_2015
union
SELECT cod_catastro, id_uso, id_subuso, id_estruc,id_esp1, id_esp2, id_esp3, id_esp4, id_esp5, id_esp6, uso_tierra, uso, subuso, estructura, cobertura, altura, tipo_fores, subtipofor, especi1_ci, especi2_ci, especi3_ci, especi4_ci, especi5_ci, especi6_ci, codreg, codprov, codcom, superf_ha,  geo_geom as geom FROM CL_CO_2014
union
SELECT cod_catastro, id_uso, id_subuso, id_estruc,id_esp1, id_esp2, id_esp3, id_esp4, id_esp5, id_esp6, uso_tierr as uso_tierra, uso, subuso, estructura, cobertura, altura, tipo_fore as tipo_fores, subtipofo as subtipofor, especi1_ci, especi2_ci, especi3_ci, especi4_ci, especi5_ci, especi6_ci, codreg, codprov, codcom, superf_ha,  geo_geom as geom FROM CL_AR_2017
union
SELECT cod_catastro, id_uso, id_subuso, id_estruc,id_esp1, id_esp2, id_esp3, id_esp4, id_esp5, id_esp6, uso_tierra, uso, subuso, estructura, cobertura, altura, tipo_fores, subtipofor, especi1_ci, especi2_ci, especi3_ci, especi4_ci, especi5_ci, especi6_ci, codreg, codprov, codcom, superf_ha,  geo_geom as geom FROM CL_LI_2013
union
SELECT cod_catastro, id_uso, id_subuso, id_estruc,id_esp1, id_esp2, id_esp3, id_esp4, id_esp5, id_esp6, uso_tierra, uso, subuso, estructura, cobertura, altura, tipo_fores, subtipofor, especi1_ci, especi2_ci, especi3_ci, especi4_ci, especi5_ci, especi6_ci, codreg, codprov, codcom, superf_ha,  geo_geom as geom FROM CL_LL_2018
union
SELECT cod_catastro, id_uso, id_subuso, id_estruc,id_esp1, id_esp2, id_esp3, id_esp4, id_esp5, id_esp6, uso_tierra, uso, subuso, estructura, cobertura, altura, tipo_fores, subtipofor, especi1_ci, especi2_ci, especi3_ci, especi4_ci, especi5_ci, especi6_ci, codreg, codprov, codcom, superf_ha,  geo_geom as geom FROM CL_LR_2017
union
SELECT cod_catastro, id_uso, id_subuso, id_estruc,id_esp1, id_esp2, id_esp3, id_esp4, id_esp5, id_esp6, uso_tierra, uso, subuso, estructura, cobertura, altura, tipo_fores, subtipofor, especi1_ci, especi2_ci, especi3_ci, especi4_ci, especi5_ci, especi6_ci, codreg, codprov, codcom, superf_ha,  geo_geom as geom FROM CL_MA_2017
union
SELECT cod_catastro, id_uso, id_subuso, id_estruc,id_esp1, id_esp2, id_esp3, id_esp4, id_esp5, id_esp6, uso_tierra, uso, subuso, estructura, cobertura, altura, tipo_fores, subtipofor, especi1_ci, especi2_ci, especi3_ci, especi4_ci, especi5_ci, especi6_ci, codreg, codprov, codcom, superf_ha,  geo_geom as geom FROM CL_ML_2016
union
SELECT cod_catastro, id_uso, id_subuso, id_estruc,id_esp1, id_esp2, id_esp3, id_esp4, id_esp5, id_esp6, uso_tierra, uso, subuso, estructura, cobertura, altura, tipo_fores, subtipofor, especi1_ci, especi2_ci, especi3_ci, especi4_ci, especi5_ci, especi6_ci, codreg, codprov, codcom, superf_ha,  geo_geom as geom FROM CL_RM_2019
union
SELECT cod_catastro, id_uso, id_subuso, id_estruc,id_esp1, id_esp2, id_esp3, id_esp4, id_esp5, id_esp6, uso_tierra, uso, subuso, estructura, cobertura, altura, tipo_fores, subtipofor, especi1_ci, especi2_ci, especi3_ci, especi4_ci, especi5_ci, especi6_ci, codreg, codprov, codcom, superf_ha,  geo_geom as geom FROM CL_TA_2016
union
SELECT cod_catastro, id_uso, id_subuso, id_estruc,id_esp1, id_esp2, id_esp3, id_esp4, id_esp5, id_esp6, uso_tierra, uso, subuso, estructura, cobertura, altura, tipo_fores, subtipofor, especi1_ci, especi2_ci, especi3_ci, especi4_ci, especi5_ci, especi6_ci, codreg, codprov, codcom, superf_ha,  geo_geom as geom FROM CL_VS_2019
)
select *
--cod_catastro,
--count(*) as npolygon,
--sum(superf_ha) as sup_c
--sum(st_area(st_transform(geo_geom,'EPSG:4326','EPSG:32718'))/10000) as sup_g
from aa
--group by cod_catastro, id_uso, id_subuso, id_estruc,id_esp1, id_esp2, id_esp3, id_esp4, id_esp5, id_esp6
''')


In [ ]:
con.sql('''
copy
(select cod_catastro, id_uso, id_subuso, id_estruc,id_esp1, id_esp2, id_esp3, id_esp4, id_esp5, id_esp6, uso_tierra, uso, subuso, estructura, cobertura, altura, tipo_fores, subtipofor, especi1_ci, especi2_ci, especi3_ci, especi4_ci, especi5_ci, especi6_ci, codreg, codprov, codcom, superf_ha, st_astext(geom) as wkt from catastro_bn)
 TO 'CAT_BN_RP.parquet' (FORMAT PARQUET)
 ''')